# Second-Order Instrument Response

This notebook calculates the response of a second-order instrument to input signals composed of DC, cosine, and sine terms.

**Transfer Function:**
$$H(s) = \frac{K\omega_n^2}{s^2 + 2\xi\omega_n s + \omega_n^2}$$

In [ ]:
import numpy as np

## 1. Input Parameters

In [ ]:
K = 2          # Static sensitivity
xi = 1.6       # Damping factor (ξ)
wn = 5         # Natural frequency ωn (rad/sec)

# Input signal: list of (amplitude, frequency, type)
# type: 'dc', 'cos', 'sin'
signal = [
    (5, 0, 'dc'),       # 5 (DC term)
    (4, 100, 'cos'),    # 4·cos(100t)
    (2, 200, 'sin'),    # 2·sin(200t)
]

# Display input signal
terms = []
for amp, freq, typ in signal:
    if typ == 'dc': terms.append(f"{amp}")
    elif typ == 'cos': terms.append(f"{amp}·cos({freq}t)")
    else: terms.append(f"{amp}·sin({freq}t)")
print(f"X(t) = {' + '.join(terms)}")

## 2. Transfer Function H(s)

In [ ]:
# H(s) = K·ωn² / (s² + 2ξωn·s + ωn²)
numerator = K * wn**2
coef_s1 = 2 * xi * wn  # coefficient of s
coef_s0 = wn**2        # constant term

print(f"H(s) = K·ωn² / (s² + 2ξωn·s + ωn²)")
print(f"H(s) = {K}·({wn})² / (s² + 2({xi})({wn})·s + ({wn})²)")
print(f"H(s) = {numerator} / (s² + {coef_s1}s + {coef_s0})")

## 3. Frequency Response H(jω)

In [ ]:
# Substitute s = jω
# H(jω) = numerator / ((jω)² + coef_s1·(jω) + coef_s0)
# H(jω) = numerator / (-ω² + coef_s1·jω + coef_s0)

print(f"Substitute s → jω:")
print(f"H(jω) = {numerator} / ((jω)² + {coef_s1}(jω) + {coef_s0})")
print(f"H(jω) = {numerator} / (-ω² + {coef_s1}jω + {coef_s0})")

## 4. Evaluate H(jω) for Each Frequency

In [ ]:
results = []

for amp, w, typ in signal:
    print(f"{'='*60}")
    print(f"ω = {w} rad/sec ({typ}, amplitude = {amp})")
    print(f"{'='*60}")
    
    # Denominator: (-ω² + coef_s0) + j(coef_s1·ω)
    real_part = -w**2 + coef_s0
    imag_part = coef_s1 * w
    
    print(f"H({w}j) = {numerator} / (-({w})² + {coef_s1}j({w}) + {coef_s0})")
    print(f"H({w}j) = {numerator} / ({real_part} + {imag_part}j)")
    
    if w == 0:
        # DC case: H(0) = numerator / coef_s0 = K
        H_mag = numerator / coef_s0
        H_phase = 0
        print(f"H(0) = {numerator}/{coef_s0} = {H_mag}")
    else:
        # Complex division
        denominator = complex(real_part, imag_part)
        H = numerator / denominator
        H_mag = abs(H)
        H_phase = np.arctan2(H.imag, H.real)
        
        # Show magnitude calculation
        den_mag = np.sqrt(real_part**2 + imag_part**2)
        print(f"\n|H({w}j)| = {numerator} / √(({real_part})² + ({imag_part})²)")
        print(f"|H({w}j)| = {numerator} / √{real_part**2 + imag_part**2}")
        print(f"|H({w}j)| = {numerator} / {den_mag:.4f}")
        print(f"|H({w}j)| = {H_mag:.6f}")
        
        # Show phase calculation
        print(f"\nφ = arctan({imag_part}/{real_part})")
        print(f"φ = arctan({imag_part/real_part:.6f})")
        print(f"φ = {H_phase:.4f} rad")
    
    results.append((amp, w, typ, H_mag, H_phase))
    print()

## 5. Output Signal

In [ ]:
print("Output signal Y(t):")
print()

output_terms = []
for amp, w, typ, H_mag, H_phase in results:
    out_amp = amp * H_mag
    
    if typ == 'dc':
        print(f"DC: {amp} × {H_mag:.6f} = {out_amp:.6f}")
        output_terms.append(f"{out_amp:.5f}")
    elif typ == 'cos':
        print(f"cos({w}t): {amp} × {H_mag:.6f} × cos({w}t + ({H_phase:.4f}))")
        print(f"        = {out_amp:.6f} × cos({w}t + ({H_phase:.4f}))")
        output_terms.append(f"{out_amp:.5f}·cos({w}t + ({H_phase:.2f}))")
    else:  # sin
        print(f"sin({w}t): {amp} × {H_mag:.6f} × sin({w}t + ({H_phase:.4f}))")
        print(f"        = {out_amp:.6f} × sin({w}t + ({H_phase:.4f}))")
        output_terms.append(f"{out_amp:.5f}·sin({w}t + ({H_phase:.2f}))")

print()
print("="*60)
print(f"Y(t) = {' + '.join(output_terms)}")
print("="*60)